In [ ]:
%pylab inline
from power_spectra import *
from scipy.special import jn, jn_zeros
from scipy.interpolate import interp1d
from hankel_transform import *
from astropy.cosmology import Planck13 #use Planck15 if you can
import astropy.units as u
from matplotlib import rc
rc('text', usetex=False)
import scipy
from scipy.interpolate import interp1d
import fastpt as fpt

from astropy.table import Table, hstack,join,vstack

from scipy.odr import ODR, Model, Data, RealData
from scipy.special import eval_legendre as legendre
from mcfit import P2xi
from scipy.special import gamma

from scipy.interpolate import interp1d

import seaborn as sns
import colorsys
import matplotlib.colors as mc

colors=sns.color_palette()
markers=['o','^','>','<','v','s']

In [ ]:
def chisq_dat(p,rp,wgg,wgm,cov_inv_gg,cov_inv_gm,wgg_func,wgm_func,fz,xi,do_DS,bg1):
    bg=np.atleast_1d(p[0])
    if bg1 is None:
        bg1=bg
    A_I=np.atleast_1d(p[1])
    njk=100
    hf=(njk-1.)/(njk-2.-len(rp))
    wgg_model=wgg_func(bg=np.atleast_1d(bg),bg2=np.atleast_1d(bg1),f=fz,xi=xi)
    beta2=-3
    if do_DS: #IA
        beta2=-1
    wgm_model=wgm_func(bg=bg,f=fz,xi=xi,do_DS=do_DS,beta2=beta2)*A_I
    loss_gg=wgg-wgg_model[0]
    loss_gm=wgm-wgm_model[0]
    chisq=loss_gg@(cov_inv_gg@loss_gg)
    chisq+=loss_gm@(cov_inv_gm@loss_gm)
    chisq/=hf
    print(hf)
    return .5*chisq

In [ ]:
def dat_cut(dat={},rp_max=65,rp_min=5,njk=100,diag_cov=False):
    dc={}
    x=dat[0]['1bin_value']>rp_min
    x*=dat[0]['1bin_value']<rp_max
    for i in dat.keys():
        dc[i]=dat[i][x]
    cov_gm,corr_gm=cc.dict_cov(dic=dc,var='6e',stack=1)
    cov_gg,dc['corr_gg']=cc.dict_cov(dic=dc,var='4W',stack=1)
    if diag_cov:
        cov_gm=np.diag(np.diag(cov_gm))
        cov_gg=np.diag(np.diag(cov_gg))
    hf=1 #(njk-1.)/(njk-2.-x.sum()) #we are using t-dist
    dc['cov_gm']=cov_gm*99.*99./100*hf
    dc['cov_gg']=cov_gg*99.*99./100*hf
    dc['corr_gm']=corr_gm
    dc['err']=None
    try:
        dc['cov_inv_gm']=np.linalg.inv(dc['cov_gm'])
        dc['cov_inv_gg']=np.linalg.inv(dc['cov_gg'])
    except Exception as err:
        print(err)
        dc['err']=err
    return dc

In [ ]:
def w_to_DS(rp=[],w=[],**kwargs):
    return 2.*integrate.cumtrapz(w*rp,x=rp,initial=0)/rp**2-w

def w_to_ups(rp,w,r0):
    DS=w_to_DS(rp,w)
    ups=DS_to_ups(rp,DS,r0)
    return ups

def w_to_ups_jk(dat={},col='6e',r0=0.2,njk=100):
    rp=dat[0]['1bin_value']
    uk={}
    for i in np.arange(njk):
        uk[i]=w_to_ups(rp=rp,w=dat[i][col],r0=r0)
        dat[i][col]=uk[i]
    uk=jk_mean(uk,njk=njk)
    i='data'
    dat[i][col]=uk['jk']
    return dat

def DS_to_ups(rp,DS,r0):
#     DS_intp=interp1d(np.log(rp),np.log(DS),bounds_error=True,kind='linear')
    # DS0=np.exp(DS_intp(np.log(r0)))
    DS_intp=interp1d(rp,DS,bounds_error=True,kind='linear')
    DS0=DS_intp(r0)
    ups=DS-DS0*(r0**2)/rp**2
    x=rp<r0
    ups[x]*=0
    return ups#[rp>r0]

In [ ]:
def fit_dat(home='',dataset='',corr='fp',z=np.atleast_1d([0.27]),rp_max=30,rp_min=5,njk=100,lbl='',use_chisq=False,diag_cov=False,bg1=None,kappa_ups=False):
    kargs=locals()
    dat0=jk_read(home=hm,dataset=fn,njk=njk)
    if kappa_ups:
        dat0=w_to_ups_jk(dat=dat0,njk=njk)
    dat=dat_cut(dat=dat0,rp_max=rp_max,rp_min=rp_min,njk=njk,diag_cov=diag_cov)
    if dat['err'] is not None:
        return dat
    pk,k=PS.camb_pk(z=z)
    rp=dat[0]['1bin_value']
    w_th=Projected_Corr_RSD(rp=rp,pi_max=100,k=k) #####implement binning
    xi0=w_th.get_xi(pk=pk)
    f=np.atleast_1d(cosmo2.f_z(z=z))
    wgp_f0=wgp_f[z[0]]
    
    if corr=='fp':
        do_DS=False
        wgp_f0/=3
        if kappa_ups:
            do_DS=True
    if corr=='ia':
        do_DS=True
      
    p0=[1,1.*wgp_f0]
    params={}
    bg1_i=None
    for i in np.arange(njk):
        if bg1 is not None:
            bg1_i=bg1[i]
        if use_chisq:
            pm=minimize(chisq_dat,p0,args=(rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm'], w_th.wgg_calc,w_th.wgm_calc,f,xi0,do_DS,bg1_i) ,)
        else:
            pm=minimize(t_dist,p0,args=(rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm'], w_th.wgg_calc,w_th.wgm_calc,f,xi0,do_DS,bg1_i) ,)
        params[i]=pm.x
        params[i][1]/=wgp_f0
    params=jk_mean(params,njk=njk)
    chisq_jk=2*chisq_dat([params['jk'][0],params['jk'][1]*wgp_f0],rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm'], w_th.wgg_calc,w_th.wgm_calc,f,xi0,do_DS,bg1_i)
    chisq_jk_gg=2*chisq_dat([params['jk'][0],params['jk'][1]*wgp_f0],rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm']*0, w_th.wgg_calc,w_th.wgm_calc,f,xi0,do_DS,bg1_i)
    chisq_jk_gm=2*chisq_dat([params['jk'][0],params['jk'][1]*wgp_f0],rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg']*0,dat['cov_inv_gm'], w_th.wgg_calc,w_th.wgm_calc,f,xi0,do_DS,bg1_i)
    for i in ['data','data0']:
        if bg1 is not None:
            bg1_i=bg1[i]
        if use_chisq:
            pm=minimize(chisq_dat,p0,args=(rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm'], w_th.wgg_calc,w_th.wgm_calc,f,xi0,do_DS,bg1_i) ,)
        else:
            pm=minimize(t_dist,p0,args=(rp,dat[i]['4W'],dat[i]['6e'],dat['cov_inv_gg'],dat['cov_inv_gm'], w_th.wgg_calc,w_th.wgm_calc,f,xi0,do_DS,bg1_i) ,)
        params[i]=pm.x
        params[i][1]/=wgp_f0
    out={}
    out['dat']=dat0
    out['dat_cut']=dat
    out['params']=params
    out['chisq']=chisq_jk
    out['chisq_gg']=chisq_jk_gg
    out['chisq_gm']=chisq_jk_gm
    
    rp=dat0[0]['1bin_value']
    w_th=Projected_Corr_RSD(rp=rp,pi_max=100,k=k) #####implement binning
    xi0=w_th.get_xi(pk=pk)
    bg=np.atleast_1d(params['jk'][0])
    bg2=bg
    A_I=np.atleast_1d(params['jk'][1])
    if bg1 is not None:
        bg2=bg1['jk']
    out['wgg_model']=w_th.wgg_calc(bg=np.atleast_1d(bg),f=f,xi=xi0,bg2=bg2)[0]
    out['wgm_model']=w_th.wgm_calc(bg=bg2,f=f,xi=xi0,do_DS=do_DS)[0]*wgp_f0*A_I
    out['f']=f
    out['wgp_f']=wgp_f0
    out['corr']=corr
    out['rp_max']=rp_max
    out['rp_min']=rp_min
    out['label']=lbl
    out['z']=z
    if corr=='fp' and not kappa_ups:
        kargs['kappa_ups']=True
        out['wgp']=fit_dat(**kargs)
    return out

In [ ]:
def lighten_color(color, amount=0.2):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    c = colorsys.rgb_to_hls(*mc.to_rgb(color))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

def plot_corrs(datasets={},rp_pow=1,wgp_ylim=None,wgg_ylim=None,legend_loc=(.8, 1.0),legend_col=1,
               b_x=0.2,A_x=0.2,b_y=100,db_y=15,A_y=-1,dA_y=0.6,split_A_x=None,dA_x=10,ignore_wgg=False,
               drp=0.03,xlim=(0.1,110),text_info=None,ti_x=0,ti_y=0,plot_1wgg=False,no_legend=False,
              fname=None,text_fontsize=30):
    if ignore_wgg:
        wgg_fig,wgg_axes =subplots(nrows=1, ncols=1, sharex=True, sharey=False,figsize=(15,10))
        wgp_plot=wgg_axes
    else:
        wgg_fig,wgg_axes =subplots(nrows=2, ncols=1, sharex=True, sharey=False,figsize=(15,20))
    #     wgp_fig,wgp_axes =subplots(nrows=1, ncols=1, sharex=True, sharey=False)
        wgg_plot=wgg_axes[0]
        wgp_plot=wgg_axes[1]
     
    ik=0
    for k in datasets.keys():
        color=colors[ik%len(colors)]
        marker=markers[ik%len(colors)]
        out=datasets[k]
        lbl=out['label']
        corr=out['corr']
        y_label=labels['fp']
        tl='\lambda'
        if corr=='ia':
            y_label=labels['ia']
            tl='I'
        rp=out['dat'][0]['1bin_value']
        frp=rp**rp_pow
        print(k,'chisq :',out['chisq'],out['chisq_gg'],out['chisq_gm'])
        if plot_1wgg and ik>0:
            pass
        elif not ignore_wgg:
            wgg_plot.errorbar(rp*(1+ik*drp),frp*out['dat']['data']['4W'],frp*out['dat']['data']['5W_err'],fmt=marker,color=color,label=lbl,mfc='white',mew=3,mec=color,zorder=1)
            wgg_plot.plot(rp*(1+ik*drp),frp*out['wgg_model'],ls='-',color=color,zorder=0)
            wgg_plot.set_xlabel(r'$r_p$ [Mpc/h]')
            wgg_plot.set_ylabel(labels['gg'])
            if rp_pow>0:
                wgg_plot.set_ylabel(r'$r_p$'+labels['gg']+'$^'+str(rp_pow+1)+'$')
            wgg_plot.text(x=b_x,y=b_y-ik*db_y,s='b='+round_significant_digit(out['params']['jk'][0],out['params']['jk_err'][0]),color=color,zorder=100,
                         bbox=dict(facecolor=lighten_color(color),#'lightgrey', 
                                   alpha=1,ec='white',lw=0),fontsize=text_fontsize)
            if ik==0:
                if wgg_ylim is None:
                    wgg_plot.vlines(out['rp_min'],(frp*out['dat']['data']['4W']).min(),(frp*out['dat']['data']['4W']).max(),color='c',linestyle=':')
                    wgg_plot.vlines(out['rp_max'],(frp*out['dat']['data']['4W']).min(),(frp*out['dat']['data']['4W']).max(),color='c',linestyle=':')
                else:
                    wgg_plot.vlines(out['rp_min'],min(wgg_ylim),max(wgg_ylim),color='c',linestyle=':')
                    wgg_plot.vlines(out['rp_max'],min(wgg_ylim),max(wgg_ylim),color='c',linestyle=':')
            wgg_plot.set_ylim(wgg_ylim)
                
        wgp_plot.errorbar(rp*(1+ik*drp),frp*out['dat']['data']['6e'],frp*out['dat']['data']['7e_err'],fmt=marker,label=lbl,color=color,mfc='white',mew=3,mec=color,zorder=1)
        wgp_plot.plot(rp*(1+ik*drp),frp*out['wgm_model'],'-',color=color,zorder=0)
        if split_A_x is not None:
            if ik==split_A_x:
                A_x=A_x+dA_x
                A_y=A_y+dA_y*ik
        wgp_plot.text(x=A_x,y=A_y-dA_y*ik,s='$A_'+tl+'$='+round_significant_digit(out['params']['jk'][1],out['params']['jk_err'][1]),color=color,zorder=100,
                     bbox=dict(facecolor=lighten_color(color),#'lightgrey', 
                               alpha=1,ec='white',lw=0),fontsize=text_fontsize)
        if ik==0:
            if wgp_ylim is None:
                wgp_plot.vlines(out['rp_min'],(frp*out['dat']['data']['6e']).min(),(frp*out['dat']['data']['6e']).max(),color='c',linestyle=':',zorder=-1)
                wgp_plot.vlines(out['rp_max'],(frp*out['dat']['data']['6e']).min(),(frp*out['dat']['data']['6e']).max(),color='c',linestyle=':',zorder=-1)
            else:
                wgp_plot.vlines(out['rp_min'],min(wgp_ylim),max(wgp_ylim),color='c',linestyle=':',zorder=-1)
                wgp_plot.vlines(out['rp_max'],min(wgp_ylim),max(wgp_ylim),color='c',linestyle=':',zorder=-1)
        wgp_plot.set_xlabel(r'$r_p$ [Mpc/h]')
        wgp_plot.set_ylabel(y_label)
        if rp_pow>0:
            print(r'$r_p$'+y_label+'$^'+str(rp_pow+1)+'$')
            wgp_plot.set_ylabel(r'$r_p$'+y_label+'$^'+str(rp_pow+1)+'$')
            wgp_plot.hlines(0,0.001,200,color='k',linestyle='--',lw=2,zorder=-1)
        #wgp_plot.legend()
      
        wgp_plot.set_xscale('log')
        wgp_plot.set_ylim(wgp_ylim)
        wgp_plot.set_xlim(xlim)
        if rp_pow==0:
            wgp_plot.set_yscale('log')
            wgg_plot.set_yscale('log')
        ik+=1
    if not no_legend:
        if plot_1wgg or ignore_wgg:
            wgp_plot.legend(loc='upper right',bbox_to_anchor=legend_loc,ncol=legend_col)
        else:
            wgg_plot.legend(loc='upper left',bbox_to_anchor=legend_loc,ncol=legend_col)


    if text_info is not None:
        if ignore_wgg:
            t_plot=wgp_plot
        else:
            t_plot=wgg_plot
        t_plot.text(x=ti_x,y=ti_y,s=text_info,color='k',zorder=100,
                     bbox=dict(facecolor='white', alpha=1,ec='white',lw=0)
                          )

    if fname is not None:
        fname=fig_home+fname
        wgg_fig.savefig(fname,bbox_inches='tight',fig_format=fig_format)
    wgg_fig.show()
    return 0

In [ ]:
DI={}

In [ ]:
cosmo=Planck13.clone()#we want h=1 for some comparisons

In [ ]:
cosmo.Ob0

In [ ]:
kmax=30
kmin=.8e-3
k_smooth=1
nk=5000
rmin=1
rmax=100
non_linear=1

r_bins=np.logspace(-1,np.log10(100),21) #np.logspace(0,2,11)

In [ ]:
cosmo_fid=dict({'h':cosmo.h,'Omb':cosmo.Ob0,'Omd':cosmo.Om0-cosmo.Ob0,'s8':0.817,'Om':cosmo.Om0,
                'As':2.12e-09,'mnu':cosmo.m_nu[-1].value,'Omk':cosmo.Ok0,'tau':0.06,'ns':0.965,'w':-1,'wa':0})
pk_params={'non_linear':non_linear,'kmax':kmax,'kmin':kmin,'nk':nk}
pk_params_lin={'non_linear':0,'kmax':kmax,'kmin':kmin,'nk':nk}

In [ ]:
PS=Power_Spectra(cosmo_params=cosmo_fid,pk_params=pk_params)
PS_lin=Power_Spectra(cosmo_params=cosmo_fid,pk_params=pk_params_lin)

In [ ]:
z_min=0.16
z_max=0.36
z_mean=0.27

In [ ]:
pk,kh =PS.class_pk(z=[z_mean])

In [ ]:
#growth factor
PS.DZ_int(z=[0,z_mean])

In [ ]:
z0=0.0
pk0_lin,kh0 = PS_lin.class_pk(z=[z0])

In [ ]:
#Plot some stuff to check
plot(kh0,pk0_lin[0], 'r--', label='$P_{lin}(k)$')
plot(kh,pk[0], 'g-', label='$P_{nl}(k)$')

xscale('log')
yscale('log')
xlabel('$k$', fontsize=14)
ylabel('$P(k)$', fontsize=14)
legend(loc='lower left', fontsize=12, frameon=False)

In [ ]:
#Setting up the Hankel Transform
#This part is slower. But only needs to be run once. 
#If you only need wgg, set j_nu=[0]. For wg+ (or \Delta\Sigma) use j_nu=[2]
%time HT=hankel_transform(rmin=rmin,rmax=rmax,kmax=kmax,j_nu=[0,2,4],n_zeros=28000,kmin=kmin)
#HT=hankel_transform(rmin=1,rmax=rmax,kmax=1,j_nu=[0,2],n_zeros=2800,kmin=1.e-2)#quick test... inaccurate

In [ ]:
pk_taper=HT.taper(k=kh,pk=pk[0],large_k_lower=5,large_k_upper=kmax,low_k_lower=kmin,
                     low_k_upper=kmin*2)
#need atleast k=10 and k=1.e-3 to get decent wgg for 1-100 Mpc/h range.

In [ ]:
from scipy.integrate import quad as scipy_int1d

In [ ]:
cosmo.H(z=[0]).value

In [ ]:
def DZ_int(z=[0],cosmo=None,rtol=1.e-4,tol=1.e-5): #linear growth factor.. full integral.. eq 63 in Lahav and suto
        Ez_func=cosmo.efunc
        def intf(z):
            return (1.+z)/(cosmo.H(z=z).value**3)
        dz=np.zeros_like(z,dtype='float32')
        inf=np.inf
        j=0
        for i in z:
            dz[j]+=cosmo.H(i).value*scipy_int1d(intf,i,inf,epsrel=rtol,epsabs=tol)[0]
            j=j+1
        dz=dz*2.5*cosmo.Om0*cosmo.H0**2
        return dz/dz[0] #check for normalization

In [ ]:
b_g=1.8
A_I=4.5
C1_rhoC=0.0134
Om=cosmo.Om0
Dz=DZ_int(z=np.append([0],z_mean),cosmo=cosmo)# Dz is normalized to 1 at lowest z. 
                                    #Hence appending 0 in front to get Dz normalized to 1 at z=0
Dz=Dz[1]/Dz[0]
wgg_f=b_g**2
wgp_f=b_g*A_I*C1_rhoC*Om/Dz
wpp_f=(A_I*C1_rhoC*Om/Dz)**2

In [ ]:
r_gg,wgg=HT.projected_correlation(k_pk=kh,pk=pk[0]*wgg_f,j_nu=0)
%time r_gg,wgg_taper=HT.projected_correlation(k_pk=kh,pk=pk_taper*wgg_f,j_nu=0)

r_gp,wgp=HT.projected_correlation(k_pk=kh,pk=pk[0]*wgp_f,j_nu=2)

r_pp,wpp4=HT.projected_correlation(k_pk=kh,pk=pk[0]*wpp_f,j_nu=4)
r_pp0,wpp0=HT.projected_correlation(k_pk=kh,pk=pk[0]*wpp_f,j_nu=0)

wpp0_intp=interp1d(r_pp0,wpp0,bounds_error=False,fill_value=np.nan)
wpp=wpp4+wpp0_intp(r_pp)

In [ ]:
plot(r_gg,r_gg*wgg,label='No tapering')
plot(r_gg,r_gg*wgg_taper,label='tapering')
#plot(wgg_test['rp'],wgg_test['rp']*wgg_test['wgg']*wgg_test_fact)
xscale('log')
#yscale('log')
#ylim(1.e-2,1.e3)
xlim(1,110)
xlabel('$r_p$ [Mpc/h]')
ylabel('$w_{gg}$ [Mpc/h]')

legend()

In [ ]:
plot(r_gp,wgp,label='No tapering')
xscale('log')
yscale('log')
xlim(1,110)
xlabel('$r_p$ [Mpc/h]')
ylabel('$w_{g+}$ [Mpc/h]')
ylim(1.e-2,20)
legend()

In [ ]:
plot(r_pp,r_pp*wpp,label='No tapering')
xscale('log')
xlim(1,110)
xlabel('$r_p$ [Mpc/h]')
ylabel('$w_{++}$ [Mpc/h]')
# ylim(0,72)
legend()

In [ ]:
plot(r_gg,wgg-wgg_taper,'r')
xscale('log')
xlabel('$r_p$ [Mpc/h]')

# Covariance

In [ ]:
n_g=3.e-4 #number density.. in units of Mpc ^ -3, same as pk
g_shot_noise=1./n_g
b_g=1.8
p_g=pk[0]*wgg_f

In [ ]:
sigma_e=.36
sigma_gamma=sigma_e/1.7
n_s=n_g #shape galaxy number density 
shape_noise=sigma_gamma**2/n_s#*cosmo.H_z(z=0.27)/cosmo.c
p_gk=pk[0]*wgp_f
p_kk=pk[0]*wpp_f

In [ ]:
#lensing contribution
l=np.unique(np.int32(kh*cosmo.comoving_transverse_distance(z_mean).value))
l,cl=PS.kappa_cl(zs1=[z_mean],zs2=[z_mean],p_zs1=[1],p_zs2=[1],pk_func=PS.class_pk,zl_max=z_mean,l=l)
p_kappa=cl*cosmo.comoving_transverse_distance(z_mean)**3 / 2 #this is an approximation
k_kappa=l/cosmo.comoving_transverse_distance(z_mean)
p_kappa_int=interp1d(k_kappa,p_kappa,bounds_error=False, fill_value=0)
p_kappa=p_kappa_int(kh)

In [ ]:
plot(kh,p_kappa,'--')
plot(kh,p_kk)
loglog()

In [ ]:
area=8000 #sq. degrees
area_comoving=area*(np.pi/180)**2*cosmo.comoving_distance(z=z_mean)**2
L_W=cosmo.comoving_distance(z=z_max)-cosmo.comoving_distance(z=z_min)
vol=area_comoving*L_W
vol=vol.value
area_comoving=area_comoving.value

In [ ]:
p_g_cov=p_g+g_shot_noise
p_kk_cov=p_kk+p_kappa+shape_noise
p_gk_cov=p_gk

In [ ]:
r,cov_ggkk=HT.projected_covariance(k_pk=kh,pk1=p_g_cov,pk2=p_kk_cov,j_nu=2)
r_re,cov_ggkk_re=HT.bin_cov(r=r,cov=cov_ggkk,r_bins=r_bins)
corr=HT.corr_matrix(cov=cov_ggkk_re)

In [ ]:
r,cov_gkgk=HT.projected_covariance(k_pk=kh,pk1=p_gk_cov,pk2=p_gk_cov,j_nu=2)
r_re,cov_gkgk_re=HT.bin_cov(r=r,cov=cov_gkgk,r_bins=r_bins)
corr=HT.corr_matrix(cov=cov_gkgk_re)

In [ ]:
cov_final=(cov_ggkk_re+cov_gkgk_re)/area_comoving
corr=HT.corr_matrix(cov=cov_final)
errors=HT.diagonal_err(cov=cov_final)

In [ ]:
lowz_dat=np.genfromtxt('./test_Dat/lowz_full.dat',names=True)
lowz_cov=np.genfromtxt('./test_Dat//lowz_wgp_cov.dat')
lowz_corr=HT.corr_matrix(cov=lowz_cov)

In [ ]:
pcolor(r_re,r_re,corr,vmin=-1,vmax=1,cmap='seismic')
colorbar()
xscale('log')
yscale('log')
xlim(HT.rmin,HT.rmax)
ylim(HT.rmin,HT.rmax)
# colorbar()
show()
pcolor(lowz_dat['rp'],lowz_dat['rp'],corr,vmin=-1,vmax=1,cmap='seismic')
colorbar()
xscale('log')
yscale('log')
xlim(HT.rmin,HT.rmax)
ylim(HT.rmin,HT.rmax)
# colorbar()

In [ ]:
plot(r_gp,wgp)
plot(r_re,errors,'--')
xscale('log')
yscale('log')
xlim(1,110)

In [ ]:
plot(r_gp,wgp)
plot(lowz_dat['rp'],lowz_dat['wgp'])
plot(lowz_dat['rp'],np.sqrt(np.diag(lowz_cov)))
plot(r_re,errors)
xscale('log')
yscale('log')
xlim(1,110)
ylim(9.e-3,2)